In [122]:
import pandas as pd
import numpy as np  
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity


# GET HOTELS IN THE SAME CITIES THE CHANNEL ALREADY HAS A FOOT IN

In [ ]:
hotel_city_chanel_combin_extract  = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\other\\hotel_city_chanel_combin_extract.csv')
hotel_city_chanel_combin_extract.dropna(inplace=True)
hotel_city_chanel_combin_extract

In [ ]:
print(hotel_city_chanel_combin_extract['Canal_ID'].nunique(), 
        hotel_city_chanel_combin_extract['Cidade_ID'].nunique() ,
            hotel_city_chanel_combin_extract['Hotel_ID'].nunique() )

In [125]:
def get_unseen_hotels(df, canal_id):
    # Ensure canal_id is an integer
    if not isinstance(canal_id, int):
        raise ValueError(f"Expected an integer for canal_id, but got {type(canal_id)}.")

    # Get the Cidade_IDs corresponding to the given Canal_ID
    cidade_ids = df.loc[df['Canal_ID'] == canal_id, 'Cidade_ID'].unique()

    # Get all Hotel_IDs in the corresponding Cidade_IDs
    hotels_in_cidade = df.loc[df['Cidade_ID'].isin(cidade_ids), 'Hotel_ID'].unique()

    # Get the Hotel_IDs already associated with the given Canal_ID
    seen_hotels = df.loc[df['Canal_ID'] == canal_id, 'Hotel_ID'].unique()

    # Filter out seen hotels
    unseen_hotels = [hotel for hotel in hotels_in_cidade if hotel not in seen_hotels]

    # Return unseen hotels as a DataFrame column, ensuring uniqueness
    unseen_hotels_df = pd.DataFrame({'Unseen_Hotel_ID': unseen_hotels}).drop_duplicates()

    return unseen_hotels_df

In [126]:
canal_target = 1245

In [127]:
suggested_hotels = get_unseen_hotels(hotel_city_chanel_combin_extract, canal_target)

In [ ]:
suggested_hotels

In [129]:
original_hotels = hotel_city_chanel_combin_extract[hotel_city_chanel_combin_extract['Canal_ID'] == canal_target].drop_duplicates()

In [130]:
original_hotels = original_hotels.drop(columns=['Canal_ID'])

In [131]:
original_cities = original_hotels['Cidade_ID'].unique()

In [ ]:
original_cities

In [133]:
suggested_hotels = pd.merge(suggested_hotels, 
                       hotel_city_chanel_combin_extract[['Cidade_ID', 'Hotel_ID']].drop_duplicates(), 
                       left_on='Unseen_Hotel_ID', 
                       right_on='Hotel_ID',
                       how='left').drop(columns=['Hotel_ID']).drop_duplicates()

In [ ]:
suggested_hotels

# GET HOTEL FEATURES

In [135]:
data_lake_prd_314410_cz_hoteis = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\lookups\\data-lake-prd-314410.cz.hoteis.csv')

data_lake_prd_314410_cz_hoteis = data_lake_prd_314410_cz_hoteis[['Hotel_ID', 'Estrelas', 'Quartos', 'CategoriaHotel', 'Moeda', 'Cidade_ID']]

data_lake_prd_314410_cz_cidades = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\lookups\\data-lake-prd-314410.cz.cidades.csv')

data_lake_prd_314410_cz_cidades = data_lake_prd_314410_cz_cidades[['Cidade_ID', 'Pais']]

merged_data = pd.merge(data_lake_prd_314410_cz_hoteis, 
                       data_lake_prd_314410_cz_cidades, 
                       on='Cidade_ID', 
                       how='left')

merged_data.dropna(subset=['Pais'], inplace=True)

merged_data.drop(columns=['Cidade_ID'], inplace=True)


In [ ]:
data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\other\\data-lake-prd-314410.cz.pull-pesquisas_average_price_hotel_currency.csv')
data_lake_prd_314410_cz_moedas = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\lookups\\data-lake-prd-314410.cz.moedas.csv')

data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency = pd.merge(data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency, 
                     data_lake_prd_314410_cz_moedas[['Moeda_ID', 'Cotacao_USD']], 
                     on='Moeda_ID', 
                     how='left')


data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency['weighted_mean_diaria_usd'] = \
    data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency['weighted_mean_diaria'] * \
        data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency['Cotacao_USD']

merged_data = pd.merge(merged_data, 
                     data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency[['Hotel_ID', 'Cotacao_USD', 'weighted_mean_diaria_usd']], 
                     on='Hotel_ID', 
                     how='left')

data_lake_prd_314410_cz_pull_pesquisas_estadia_x_reservas_volume = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\other\\data-lake-prd-314410.cz.pull-pesquisas_estadia_x_reservas_volume.csv')

merged_data = pd.merge(merged_data, 
                     data_lake_prd_314410_cz_pull_pesquisas_estadia_x_reservas_volume, 
                     on='Hotel_ID', 
                     how='left')

data_lake_prd_314410_cz_pull_pesquisas_estadia_x_reservas_x_price_receitas = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\other\\data-lake-prd-314410.cz.pull-pesquisas_estadia_x_reservas_x_price_receitas.csv')
data_lake_prd_314410_cz_pull_pesquisas_estadia_x_reservas_x_price_receitas

merged_data = pd.merge(merged_data, 
                     data_lake_prd_314410_cz_pull_pesquisas_estadia_x_reservas_x_price_receitas, 
                     on='Hotel_ID', 
                     how='left')

merged_data['total_reservas_estadia_diaria'] = \
    merged_data['total_reservas_estadia_diaria'] * \
        merged_data['Cotacao_USD']

merged_data.drop(columns=['Moeda'], inplace=True)

merged_data.drop(columns=['Cotacao_USD'], inplace=True)

merged_data.dropna(inplace=True)

merged_data

In [137]:
merged_data.drop(columns=['Pais', 'CategoriaHotel'], inplace=True)

In [138]:
hotel_characteristics = merged_data

# Merge original and suggested hotels with their characteristics

In [ ]:


# Merge original and suggested hotels with their characteristics
features_to_scale = ['Estrelas', 'Quartos', 'weighted_mean_diaria_usd', 
                     'total_reservas_estadia', 'total_reservas_estadia_diaria']

# Merge original hotels
original_hotels_features = pd.merge(
    original_hotels,
    hotel_characteristics,
    on='Hotel_ID',
    how='left'
)

# Merge suggested hotels
suggested_hotels_features = pd.merge(
    suggested_hotels,
    hotel_characteristics,
    left_on='Unseen_Hotel_ID',
    right_on='Hotel_ID',
    how='left'
)

# Scale features
scaler = StandardScaler()
original_hotels_features[features_to_scale] = scaler.fit_transform(original_hotels_features[features_to_scale])
suggested_hotels_features[features_to_scale] = scaler.transform(suggested_hotels_features[features_to_scale])

print("Original Hotel Features:\n", original_hotels_features.head())
print("Suggested Hotel Features:\n", suggested_hotels_features.head())

In [ ]:
suggested_hotels_features

In [ ]:


# List to store results
all_city_similarities = []

# Loop through each city
for city_id in original_cities:
    # Filter hotels by city
    city_original_hotels = original_hotels_features[original_hotels_features['Cidade_ID'] == city_id].dropna()
    city_suggested_hotels = suggested_hotels_features[suggested_hotels_features['Cidade_ID'] == city_id].dropna()
    
    if city_original_hotels.empty or city_suggested_hotels.empty:
        continue

    # Extract feature matrices
    original_features = city_original_hotels[features_to_scale].values
    suggested_features = city_suggested_hotels[features_to_scale].values

    # Calculate cosine similarity
    similarity_matrix = cosine_similarity(original_features, suggested_features)

    # Create a DataFrame to hold all similarity scores
    similarity_df = pd.DataFrame(
        similarity_matrix,
        index=city_original_hotels['Hotel_ID'],
        columns=city_suggested_hotels['Unseen_Hotel_ID']
    ).reset_index()

    # Melt DataFrame to long format: one row per (original, suggested) pair
    similarity_long = similarity_df.melt(
        id_vars='Hotel_ID',  # Original Hotel ID
        var_name='Unseen_Hotel_ID',  # Suggested Hotel ID
        value_name='Similarity'  # Similarity Score
    )

    # Add the city to the DataFrame
    similarity_long['Cidade_ID'] = city_id

    # Append results
    all_city_similarities.append(similarity_long)

# Combine all city similarities
ranked_hotels = pd.concat(all_city_similarities, ignore_index=True)

# Sort by Original Hotel, City, and Similarity
ranked_hotels = ranked_hotels.sort_values(
    by=['Cidade_ID', 'Hotel_ID', 'Similarity'], ascending=[True, True, False]
).reset_index(drop=True)

print("Ranked Hotels:\n", ranked_hotels.head())



In [ ]:
ranked_hotels['Hotel_ID'].unique() 

In [ ]:
ranked_hotels['Similarity'].min()

In [ ]:
print(ranked_hotels[ranked_hotels['Hotel_ID'] == 7410.0].head(1))
print(ranked_hotels[ranked_hotels['Hotel_ID'] == 7410.0].tail(1))

In [ ]:
hotel_characteristics[hotel_characteristics['Hotel_ID'] == 7410]

In [ ]:
hotel_characteristics[hotel_characteristics['Hotel_ID'] == 7416.0     ]

In [ ]:
hotel_characteristics[hotel_characteristics['Hotel_ID'] == 17996.0   ]